# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### In here, we use the 5 columns sessionId, itemInSession, artist, length, and song which are asked in the above question. By using WHERE sessionId = 338 AND itemInSession = 4, we filtering the data and choose (sessionId, itemInSession) as our partition key in the SELECT statement below.

In [8]:
query = """
    CREATE TABLE IF NOT EXISTS history_by_session_and_item (
        sessionId int,
        itemInSession int,
        artist text, 
        length float,
        song text,
        PRIMARY KEY ((sessionId, itemInSession))
    )
""" 
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = """INSERT INTO history_by_session_and_item (sessionId, itemInSession, artist, length,song) VALUES (%s, %s, %s, %s, %s)"""
    
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (
                int(line[8]),
                int(line[3]),
                line[0],
                float(line[5]),
                line[9],
            ))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
select_music_app_history = """SELECT artist, song,length FROM history_by_session_and_item WHERE sessionId = 338 AND itemInSession = 4"""

try:
    rows = session.execute(select_music_app_history)
except Exception as e:
    print(e)

df = pd.DataFrame(list(rows))
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Here we require the 7 columns userId, sessionId, itemInSession, artist, firstName, lastName, and song which are said within the question. Since we are filtering the information by using WHERE userid = 10 AND sessionid = 182 within the SELECT statement underneath, we got to select (userId, sessionId) as our partition key. We select itemInSession as our clustering column and we sort the result by itemInSession ascending.

In [11]:
query = """
    CREATE TABLE IF NOT EXISTS history_by_user_and_session (
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        firstName text,
        lastName text,
        song text,
        PRIMARY KEY ((userid, sessionid), itemInSession)
    )
""" 
try:
    session.execute(query)
except Exception as e:
    print(e)
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    query = """
        INSERT INTO history_by_user_and_session (
            userId,
            sessionId,
            itemInSession,
            artist,
            firstName,
            lastName,
            song
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    for line in csvreader:
        session.execute(
            query,
            (
                int(line[10]),
                int(line[8]),
                int(line[3]),
                line[0],
                line[1],
                line[4],
                line[9],
            )
        )               

In [12]:
query = """
    SELECT
        itemInSession,
        artist,
        song,
        firstName,
        lastName
    FROM history_by_user_and_session
    WHERE userid = 10
        AND sessionid = 182
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
df

,iteminsession,artist,song,firstname,lastname
0,0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,1,Three Drives,Greece 2000,Sylvie,Cruz
2,2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

####  In addition to the 3 columns song, firstName, and lastName asked in the above question, we need the userId because different users may have the same firstName and lastname. We choose (song) as our partition key and sort the results by itemInSession ascending as our clustering column. And finally filtering the data using WHERE song = 'All Hands Against His Own' in the SELECT statement below

In [13]:
query = """
    CREATE TABLE IF NOT EXISTS songs_by_user (  
        userId int,
        song text,
        firstName text,
        lastName text,
        PRIMARY KEY ((song), userId)
    )
""" 
try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    query = """
        INSERT INTO songs_by_user (
            userId,
            song,
            firstName,
            lastName
        )
        VALUES (%s, %s, %s, %s)
    """
    for line in csvreader:
        session.execute(
            query,
            (
                int(line[10]),
                line[9],
                line[1],
                line[4],
            )
        )

In [15]:
query = """
    SELECT
        firstName,
        lastName
    FROM songs_by_user
    WHERE song = 'All Hands Against His Own'
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [16]:
tables = [
    "history_by_session_and_item",
    "history_by_user_and_session",
    "songs_by_user",
]

for table in tables:
    try:
        rows = session.execute(f"DROP TABLE {table}")
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()